In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
import time
import os

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
'''
CONSTANTS
'''

TRAIN_START = pd.datetime(2010, 1, 1)
TRAIN_END = pd.datetime(2016, 12, 31)
VAL_START = pd.datetime(2017, 1, 1)
VAL_END = pd.datetime(2017, 12, 31)
TEST_START = pd.datetime(2018, 1, 1)
TEST_END = pd.datetime(2018, 9, 20)

BASE_FEATURES = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
                 '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
                 '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
                 '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
                 '50', '51', '52', 'isSunday', 'isMonday', 'isTuesday', 'isWednesday', 
                 'isThursday', 'isFriday', 'isHoliday', 'TEMP_normalized',
                 'WIND_normalized', 'SNOW_normalized', 'PRCP_normalized']

SEVEN_DAY_LAG = ['Shift-1', 'Shift-2', 'Shift-3', 'Shift-4', 'Shift-5', 'Shift-6', 'Shift-7']

SEVEN_DAY_LAG_ICD8 = ['Shift-1ICD8', 'Shift-2ICD8', 'Shift-3ICD8', 'Shift-4ICD8',
                      'Shift-5ICD8', 'Shift-6ICD8', 'Shift-7ICD8']

SEVEN_DAY_LAG_ICD8OPP = ['Shift-1ICD8opp', 'Shift-2ICD8opp', 'Shift-3ICD8opp',
                         'Shift-4ICD8opp', 'Shift-5ICD8opp', 'Shift-6ICD8opp',
                         'Shift-7ICD8opp']

ONE_DAY_LAG = ['Shift-1']

ONE_DAY_LAG_ICD8 = ['Shift-1ICD8']

ONE_DAY_LAG_ICD8OPP = ['Shift-1ICD8opp']

SEVEN_DAY_LAG_FEATURES = BASE_FEATURES + SEVEN_DAY_LAG
SEVEN_DAY_LAG_ICD8_FEATURES = BASE_FEATURES + SEVEN_DAY_LAG_ICD8
SEVEN_DAY_LAG_ICD8OPP_FEATURES = BASE_FEATURES + SEVEN_DAY_LAG_ICD8OPP

ONE_DAY_LAG_FEATURES = BASE_FEATURES + ONE_DAY_LAG
ONE_DAY_LAG_ICD8_FEATURES = BASE_FEATURES + ONE_DAY_LAG_ICD8
ONE_DAY_LAG_ICD8OPP_FEATURES = BASE_FEATURES + ONE_DAY_LAG_ICD8OPP

ALL_VISITS_LABEL = "AdjCount"
ICD8_LABEL = "ICD8perFTE"
ICD8OPP_LABEL = "ICD8oppperFTE"

m_train_end = [pd.datetime(2017, 12, 31), pd.datetime(2018, 1, 31), pd.datetime(2018, 2, 28),
                       pd.datetime(2018, 3, 31), pd.datetime(2018, 4, 30), pd.datetime(2018, 5, 31),
                       pd.datetime(2018, 6, 30), pd.datetime(2018, 7, 31), pd.datetime(2018, 8, 31)]
m_test_start = [pd.datetime(2018, 1, 1), pd.datetime(2018, 2, 1), pd.datetime(2018, 3, 1),
                      pd.datetime(2018, 4, 1), pd.datetime(2018, 5, 1), pd.datetime(2018, 6, 1),
                      pd.datetime(2018, 7, 1), pd.datetime(2018, 8, 1), pd.datetime(2018, 9, 1)]
m_test_end = [pd.datetime(2018, 1, 31), pd.datetime(2018, 2, 28), pd.datetime(2018, 3, 31),
                     pd.datetime(2018, 4, 30), pd.datetime(2018, 5, 31), pd.datetime(2018, 6, 30),
                     pd.datetime(2018, 7, 31), pd.datetime(2018, 8, 31), pd.datetime(2018, 9, 20)]



In [3]:
'''
Reads our dataset into a pandas dataframe and converts the dataframe
index into datetime objects.

'''

def read_data():
    # this needs to go up one directory since all files using it are inside a subdirectory
    visits = pd.read_csv("../DatasetForMLHC.csv") 
    visits.index = pd.to_datetime(visits["Unnamed: 0"])
    visits.index = pd.DatetimeIndex(visits.index.values)
    visits = visits.reindex(visits.asfreq('d').index, fill_value=0)
    return visits

In [4]:
'''
HELPER METHODS FOR LINEAR REGRESSION
'''

def plot_weekday_weekend_outputs(model_label, X_train, X_test, y_train, y_pred, y_test,
                    X_train2, X_test2, y_train2, y_pred2, y_test2):
    fig, ax = plt.subplots(figsize=(20,20))
    
    plt.subplot(3, 1, 1)
    plt.scatter(X_train.index, y_train, marker='.', color='gray', label='training data sick visits/fte weekday')
    plt.scatter(X_test.index, y_pred, marker='.', color='red', label='predicted sick visits/fte weekday')
    plt.scatter(X_test.index, y_test, marker='.', color='green', label='actual sick visits/fte weekday')
    plt.scatter(X_train2.index, y_train2, marker='.', color='black', label='training data sick visits/fte weekend')
    plt.scatter(X_test2.index, y_pred2, marker='.', color='orange', label='predicted sick visits/fte weekend')
    plt.scatter(X_test2.index, y_test2, marker='.', color='blue', label='actual sick visits/fte weekend')
    plt.legend(loc=1)
    plt.xlabel("Date")
    plt.ylabel("Sick Visits/FTE")
    plt.title("%s: January 2010 - September 2018" % (model_label))


    plt.subplot(3, 1, 2)
    plt.scatter(X_train.index, y_train, marker='.', color='gray', label='training data sick visits/fte weekday')
    plt.scatter(X_test.index, y_pred, marker='.', color='red', label='predicted sick visits/fte weekday')
    plt.scatter(X_test.index, y_test, marker='.', color='green', label='actual sick visits/fte weekday')
    plt.scatter(X_train2.index, y_train2, marker='.', color='black', label='training data sick visits/fte weekend')
    plt.scatter(X_test2.index, y_pred2, marker='.', color='orange', label='predicted sick visits/fte weekend')
    plt.scatter(X_test2.index, y_test2, marker='.', color='blue', label='actual sick visits/fte weekend')
    plt.legend(loc=1)
    plt.xlim((pd.datetime(2017,1,1), pd.datetime(2018,9,20)))
    plt.xlabel("Date")
    plt.ylabel("Sick Visits/FTE")
    plt.title("%s: 2017 Data and 2018 Predictions" % (model_label))

    plt.subplot(3, 1, 3)
    plt.scatter(X_test.index, y_pred, marker='.', color='red', label='predicted sick visits/fte weekday')
    plt.scatter(X_test.index, y_test, marker='.', color='green', label='actual sick visits/fte weekday')
    plt.scatter(X_test2.index, y_pred2, marker='.', color='orange', label='predicted sick visits/fte weekend')
    plt.scatter(X_test2.index, y_test2, marker='.', color='blue', label='actual sick visits/fte weekend')
    plt.legend(loc=1)
    plt.xlim((pd.datetime(2018,1,1), pd.datetime(2018,9,20)))
    plt.xlabel("Date")
    plt.ylabel("Sick Visits/FTE")
    plt.title("%s: 2018 Predictions" % (model_label))
    
    plt.show()
    
    fig, ax = plt.subplots(figsize=(17,7))
    plt.scatter(X_test.index, y_pred, marker='.', color='red', label='predicted sick visits/fte weekday')
    plt.scatter(X_test.index, y_test, marker='.', color='green', label='actual sick visits/fte weekday')
    plt.scatter(X_test2.index, y_pred2, marker='.', color='orange', label='predicted sick visits/fte weekend')
    plt.scatter(X_test2.index, y_test2, marker='.', color='blue', label='actual sick visits/fte weekend')
    plt.legend(loc=1)
    plt.xlim((pd.datetime(2018,1,1), pd.datetime(2018,9,20)))
    plt.xlabel("Date")
    plt.ylabel("Sick Visits/FTE")
    plt.title("%s: 2018 Predictions" % (model_label))
    plt.show()
    
'''
Returns the total MSE of two sets of predictions that may not necessarily be the same size.

Let A be the first dataset, and B be the second dataset.

y_pred1: The predicted values for A
y_test1: The actual values for A
y_pred2: The predicted values for B
y_test2: The actual values for B

'''

def mse_weighted_avg(y_test1, y_pred1, y_test2, y_pred2):
    return (mean_squared_error(y_test2, y_pred2) * len(y_pred2) 
            + mean_squared_error(y_test1, y_pred1) * len(y_pred1)) / (len(y_pred1) + len(y_pred2))

In [5]:
'''
HELPER METHODS FOR NN AND RNN METHODS
'''

class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

def plot_history(hist):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
    plt.legend(loc=1)
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
    plt.ylim(0, 20)
    plt.legend(loc=1)

'''
Spilts the dataset into training, validation, and testing, for our dynamic,
OSH and OMA baseline, NN, and RNN models.
'''
def split_for_baseline_and_nn(X, y):
    X_train = X.loc[TRAIN_START:TRAIN_END]
    y_train = y.loc[TRAIN_START:TRAIN_END]
    X_val = X.loc[VAL_START:VAL_END]
    y_val = y.loc[VAL_START:VAL_END]
    X_test = X.loc[TEST_START:TEST_END]
    y_test = y.loc[TEST_START:TEST_END]
    return X_train, y_train, X_val, y_val, X_test, y_test

'''
Get X_train_t and X_test_t into the correct shape for the RNN model. For example:
X_train_t = np.array(X_train_scaled).reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_t = np.array(X_test_scaled).reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])
'''
def reshape_for_rnn(a):
    return np.array(a).reshape(a.shape[0], 1, a.shape[1])

'''
Prints the mean and standard deviation for an array of MSEs.
'''
def print_mse_metrics(mses):
    print("MSE and RMSE over 10 trials with standard deviation in parentheses")
    print("Average MSE: %.3f (%.3f)\nAverage RMSE: %.3f (%.3f)"
          % (np.mean(mses), np.std(mses), np.mean(np.sqrt(mses)), np.std(np.sqrt(mses))))

'''
Gets the monthly split, next, and end dates for a given index.
'''
def get_split_dates(i):
    if i > 9:
        return -1, -1, -1
    return m_train_end[i], m_test_start[i], m_test_end[i]

'''
Gets the dynamic split and next dates.
'''
def get_dynamic_split_dates():
    return m_train_end[0], m_test_start[0]

In [6]:
'''
Builds and compiles our NN model.
'''
def build_nn_model_generic(X_train, first_layer_size, second_layer_size):
    model = keras.Sequential([
    layers.Dense(first_layer_size, activation=tf.nn.relu, input_shape=[len(X_train.keys())]),
    layers.Dense(second_layer_size, activation=tf.nn.relu),
    layers.Dense(1)
    ])

    optimizer = tf.train.AdamOptimizer(1e-4)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

def compile_nn_weights_from_file(filename, X_train, first_layer_size, second_layer_size):
    model = build_nn_model_generic(X_train, first_layer_size, second_layer_size)
    model.load_weights(filename)
    optimizer = tf.train.AdamOptimizer(1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

def build_nn_model(X_train):
    return build_nn_model_generic(X_train, 64, 64)

def build_nn_model1(X_train):
    return build_nn_model_generic(X_train, 64, 32)

def build_nn_model2(X_train):
    return build_nn_model_generic(X_train, 64, 16)

def build_nn_model3(X_train):
    return build_nn_model_generic(X_train, 32, 32)

def build_nn_model4(X_train):
    return build_nn_model_generic(X_train, 32, 16)

def build_nn_model5(X_train):
    return build_nn_model_generic(X_train, 32, 8)

In [7]:
'''
Builds and compiles our RNN model.
'''
def build_rnn_model_generic(X_train, first_layer_size, second_layer_size):
    model = keras.Sequential()
    model.add(layers.LSTM(first_layer_size, activation=tf.nn.relu, input_shape=(1, X_train.shape[1]),
                kernel_initializer='lecun_uniform', return_sequences=True))
    model.add(layers.LSTM(second_layer_size, return_sequences=False))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    optimizer = tf.train.AdamOptimizer(1e-4)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

def compile_rnn_weights_from_file(filename, X_train, first_layer_size, second_layer_size):
    model = build_rnn_model_generic(X_train, first_layer_size, second_layer_size)
    model.load_weights(filename)
    optimizer = tf.train.AdamOptimizer(1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

def build_rnn_model(X_train):
    return build_rnn_model_generic(X_train, 64, 64)

def build_rnn_model1(X_train):
    return build_rnn_model_generic(X_train, 64, 32)

def build_rnn_model2(X_train):
    return build_rnn_model_generic(X_train, 64, 16)

def build_rnn_model3(X_train):
    return build_rnn_model_generic(X_train, 32, 32)

def build_rnn_model4(X_train):
    return build_rnn_model_generic(X_train, 32, 16)

def build_rnn_model5(X_train):
    return build_rnn_model_generic(X_train, 32, 8)

In [8]:
prebuilt_rnn_models = dict()

prebuilt_rnn_models["64-64"] = ['RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial3-1562305234.6405256.h5',
 'RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial1-1562303707.4653575.h5',
 'RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial4-1562305849.0060897.h5',
 'RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial2-1562304439.9532201.h5',
 'RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial0-1562302921.312181.h5']
prebuilt_rnn_models["64-32"] = ['RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial3-1562309408.0298276.h5',
 'RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial4-1562310198.6327813.h5',
 'RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial1-1562307769.6175947.h5',
 'RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial2-1562308652.2412608.h5',
 'RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial0-1562306850.1827445.h5']
prebuilt_rnn_models["64-16"] = ['RNNModels/64-16/Dynamic-RNN-AllVisits-64-16-Trial1-1562311845.4924283.h5',
 'RNNModels/64-16/Dynamic-RNN-AllVisits-64-16-Trial2-1562312736.0775466.h5',
 'RNNModels/64-16/Dynamic-RNN-AllVisits-64-16-Trial4-1562314675.5489564.h5',
 'RNNModels/64-16/Dynamic-RNN-AllVisits-64-16-Trial3-1562313718.7428586.h5',
 'RNNModels/64-16/Dynamic-RNN-AllVisits-64-16-Trial0-1562311040.2823577.h5']
prebuilt_rnn_models["32-32"] = ['RNNModels/32-32-0Lag-All/Dynamic-RNN-AllVisits-32-32-Trial0-1562316203.5722883.h5',
 'RNNModels/32-32-0Lag-All/Dynamic-RNN-AllVisits-32-32-Trial2-1562318933.3989708.h5',
 'RNNModels/32-32-0Lag-All/Dynamic-RNN-AllVisits-32-32-Trial4-1562322144.5695276.h5',
 'RNNModels/32-32-0Lag-All/Dynamic-RNN-AllVisits-32-32-Trial3-1562320836.649595.h5',
 'RNNModels/32-32-0Lag-All/Dynamic-RNN-AllVisits-32-32-Trial1-1562317741.1352634.h5']
prebuilt_rnn_models["32-16"] = ['RNNModels/32-16/Dynamic-RNN-AllVisits-32-16-Trial0-1562323427.716574.h5',
 'RNNModels/32-16/Dynamic-RNN-AllVisits-32-16-Trial1-1562324848.774883.h5',
 'RNNModels/32-16/Dynamic-RNN-AllVisits-32-16-Trial2-1562325937.957211.h5',
 'RNNModels/32-16/Dynamic-RNN-AllVisits-32-16-Trial4-1562328964.8270013.h5',
 'RNNModels/32-16/Dynamic-RNN-AllVisits-32-16-Trial3-1562327748.839671.h5']
prebuilt_rnn_models["32-8"] = ['RNNModels/32-8/Dynamic-RNN-AllVisits-32-8-Trial2-1562333011.8014703.h5',
 'RNNModels/32-8/Dynamic-RNN-AllVisits-32-8-Trial4-1562335803.0272052.h5',
 'RNNModels/32-8/Dynamic-RNN-AllVisits-32-8-Trial3-1562334331.7956104.h5',
 'RNNModels/32-8/Dynamic-RNN-AllVisits-32-8-Trial1-1562331792.8184147.h5',
 'RNNModels/32-8/Dynamic-RNN-AllVisits-32-8-Trial0-1562330429.047994.h5']
prebuilt_rnn_models["32-32R"] = ['RNNModels/32-32-0Lag-R/Dynamic-RNN-RespVisits-32-32-Trial4-1562366029.9926655.h5',
 'RNNModels/32-32-0Lag-R/Dynamic-RNN-RespVisits-32-32-Trial0-1562364481.482371.h5',
 'RNNModels/32-32-0Lag-R/Dynamic-RNN-RespVisits-32-32-Trial2-1562365257.6494977.h5',
 'RNNModels/32-32-0Lag-R/Dynamic-RNN-RespVisits-32-32-Trial3-1562365645.8746376.h5',
 'RNNModels/32-32-0Lag-R/Dynamic-RNN-RespVisits-32-32-Trial1-1562364882.4930165.h5']
prebuilt_rnn_models["32-32NR"] = ['RNNModels/32-32-0Lag-NR/Dynamic-RNN-NonRespVisits-32-32-Trial3-1562367965.7343554.h5',
 'RNNModels/32-32-0Lag-NR/Dynamic-RNN-NonRespVisits-32-32-Trial2-1562367439.946335.h5',
 'RNNModels/32-32-0Lag-NR/Dynamic-RNN-NonRespVisits-32-32-Trial0-1562366476.9159904.h5',
 'RNNModels/32-32-0Lag-NR/Dynamic-RNN-NonRespVisits-32-32-Trial4-1562368359.5355737.h5',
 'RNNModels/32-32-0Lag-NR/Dynamic-RNN-NonRespVisits-32-32-Trial1-1562366886.6321354.h5']

In [9]:
prebuilt_nn_models = dict()

prebuilt_nn_models["64-64"] = ['NNModels/64-64-0Lag-All/Dynamic-NN-AllVisits-64-64-Trial0-1562302673.171411.h5',
 'NNModels/64-64-0Lag-All/Dynamic-NN-AllVisits-64-64-Trial4-1562302927.789633.h5',
 'NNModels/64-64-0Lag-All/Dynamic-NN-AllVisits-64-64-Trial1-1562302733.186472.h5',
 'NNModels/64-64-0Lag-All/Dynamic-NN-AllVisits-64-64-Trial3-1562302863.757526.h5',
 'NNModels/64-64-0Lag-All/Dynamic-NN-AllVisits-64-64-Trial2-1562302795.191118.h5']
prebuilt_nn_models["64-32"] = ['NNModels/64-32/Dynamic-NN-AllVisits-64-32-Trial0-1562303001.808383.h5',
 'NNModels/64-32/Dynamic-NN-AllVisits-64-32-Trial3-1562303246.706902.h5',
 'NNModels/64-32/Dynamic-NN-AllVisits-64-32-Trial4-1562303335.774457.h5',
 'NNModels/64-32/Dynamic-NN-AllVisits-64-32-Trial1-1562303089.224552.h5',
 'NNModels/64-32/Dynamic-NN-AllVisits-64-32-Trial2-1562303178.582649.h5']
prebuilt_nn_models["64-16"] = ['NNModels/64-16/Dynamic-NN-AllVisits-64-16-Trial4-1562303844.51556.h5',
 'NNModels/64-16/Dynamic-NN-AllVisits-64-16-Trial1-1562303511.52243.h5',
 'NNModels/64-16/Dynamic-NN-AllVisits-64-16-Trial3-1562303691.376045.h5',
 'NNModels/64-16/Dynamic-NN-AllVisits-64-16-Trial2-1562303597.711927.h5',
 'NNModels/64-16/Dynamic-NN-AllVisits-64-16-Trial0-1562303419.257252.h5']
prebuilt_nn_models["32-32"] = ['NNModels/32-32/Dynamic-NN-AllVisits-32-32-Trial0-1562303941.1555321.h5',
 'NNModels/32-32/Dynamic-NN-AllVisits-32-32-Trial4-1562304483.8239071.h5',
 'NNModels/32-32/Dynamic-NN-AllVisits-32-32-Trial1-1562304052.071458.h5',
 'NNModels/32-32/Dynamic-NN-AllVisits-32-32-Trial3-1562304313.145748.h5',
 'NNModels/32-32/Dynamic-NN-AllVisits-32-32-Trial2-1562304159.670687.h5']
prebuilt_nn_models["32-16"] = ['NNModels/32-16/Dynamic-NN-AllVisits-32-16-Trial0-1562304609.049628.h5',
 'NNModels/32-16/Dynamic-NN-AllVisits-32-16-Trial3-1562305079.74227.h5',
 'NNModels/32-16/Dynamic-NN-AllVisits-32-16-Trial2-1562304926.2364218.h5',
 'NNModels/32-16/Dynamic-NN-AllVisits-32-16-Trial1-1562304745.71314.h5',
 'NNModels/32-16/Dynamic-NN-AllVisits-32-16-Trial4-1562305240.373695.h5']
prebuilt_nn_models["32-8"] = ['NNModels/32-8/Dynamic-NN-AllVisits-32-8-Trial4-1562306128.144107.h5',
 'NNModels/32-8/Dynamic-NN-AllVisits-32-8-Trial0-1562305457.9349172.h5',
 'NNModels/32-8/Dynamic-NN-AllVisits-32-8-Trial3-1562305935.470507.h5',
 'NNModels/32-8/Dynamic-NN-AllVisits-32-8-Trial1-1562305612.265461.h5',
 'NNModels/32-8/Dynamic-NN-AllVisits-32-8-Trial2-1562305793.3554618.h5']
prebuilt_nn_models["64-64R"] = ['NNModels/64-64-0Lag-R/Dynamic-NN-RespVisits-64-64-Trial2-1562368216.3867679.h5',
 'NNModels/64-64-0Lag-R/Dynamic-NN-RespVisits-64-64-Trial0-1562368090.3568761.h5',
 'NNModels/64-64-0Lag-R/Dynamic-NN-RespVisits-64-64-Trial3-1562368299.999485.h5',
 'NNModels/64-64-0Lag-R/Dynamic-NN-RespVisits-64-64-Trial4-1562368366.227404.h5',
 'NNModels/64-64-0Lag-R/Dynamic-NN-RespVisits-64-64-Trial1-1562368146.8525062.h5']
prebuilt_nn_models["64-64NR"] = ['NNModels/64-64-0Lag-NR/Dynamic-NN-NonRespVisits-64-64-Trial0-1562368401.0504398.h5',
 'NNModels/64-64-0Lag-NR/Dynamic-NN-NonRespVisits-64-64-Trial3-1562368511.382907.h5',
 'NNModels/64-64-0Lag-NR/Dynamic-NN-NonRespVisits-64-64-Trial4-1562368552.901161.h5',
 'NNModels/64-64-0Lag-NR/Dynamic-NN-NonRespVisits-64-64-Trial1-1562368437.986389.h5',
 'NNModels/64-64-0Lag-NR/Dynamic-NN-NonRespVisits-64-64-Trial2-1562368474.3667378.h5']

In [10]:
layer_sizes = dict()
layer_sizes["64-64"] = [64, 64]
layer_sizes["64-64R"] = [64, 64]
layer_sizes["64-64NR"] = [64, 64]
layer_sizes["64-32"] = [64, 32]
layer_sizes["64-16"] = [64, 16]
layer_sizes["32-32"] = [32, 32]
layer_sizes["32-32R"] = [32, 32]
layer_sizes["32-32NR"] = [32, 32]
layer_sizes["32-16"] = [32, 16]
layer_sizes["32-8"] = [32, 8]

In [11]:
possible_architectures = ["64-64", "64-32", "64-16", "32-32", "32-16", "32-8"]

In [12]:
possible_architectures_nn = ["64-64", "64-64R", "64-64NR"]

In [13]:
possible_architectures_rnn = ["32-32", "32-32R", "32-32NR"]